In [9]:
from __future__ import division

import os
import time

import cv2
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np

# from util import * 

In [10]:
LABELSPATH = '../../darknet/data/coco.names'
CONFIGPATH = '../../darknet/cfg/yolov3.cfg'
WEIGHTSPATH = '../yolov3.weights'

CONFIDENCE_THS = 0.5
NMS_THS = 0.3

In [11]:
LABELS = open(LABELSPATH).read().strip().split("\n")

np.random.seed(42)
COLORS = np.random.randint(0,255,size=(len(LABELS),3),dtype="uint8")

net = cv2.dnn.readNetFromDarknet(CONFIGPATH,WEIGHTSPATH)

In [12]:
def test_on_image(net,imgfile):
    image = cv2.imread(imgfile)
    (H,W) = image.shape[:2]

    output_ln = net.getLayerNames()
    output_ln = [output_ln[i[0]-1] for i in net.getUnconnectedOutLayers()]

    blob = cv2.dnn.blobFromImage(image,1/255.0,(416,416),swapRB=True,crop=False)
    net.setInput(blob)
    start = time.time()
    layer_outputs = net.forward(output_ln)
    end = time.time()

    print('[INFO] YOLO took {:.6f} secs'.format(end-start))

    return image,layer_outputs,H,W

In [13]:
def process_outputs(layer_outputs,H,W):
    boxes = []
    confidences = []
    classIDs = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if confidence > CONFIDENCE_THS:
                box = detection[0:4] * np.array([W,H,W,H])
                (center_x,center_y,width,height) = box.astype("int")

                x = int(center_x - (width/2))
                y = int(center_y - (height/2))

                boxes.append([x,y,int(width),int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes,confidences,CONFIDENCE_THS,NMS_THS)
    return idxs,boxes,confidences,classIDs

def draw_outputs(idxs,boxes,confidences,classIDs,image):
    if len(idxs) > 0:
        for i in idxs.flatten():
            (x,y) = (boxes[i][0],boxes[i][1])
            (w,h) = (boxes[i][2],boxes[i][3])

            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(image,(x,y),(x+w,y+h),color,2)
            text = "{}:{:.4f}".format(LABELS[classIDs[i]],confidences[i])
            cv2.putText(image,text,(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)
    
    return image

In [14]:
imagefile = '../images/dog-cycle-car.png'
image,layer_outputs,H,W = test_on_image(net,imagefile)
idxs,boxes,confidences,classIDs = process_outputs(layer_outputs,H,W)
output_image = draw_outputs(idxs,boxes,confidences,classIDs,image)

[INFO] YOLO took 0.441732 secs


In [15]:
cv2.imshow("",output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1